Import du modèle

In [2]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image


In [3]:
np.random.seed(3)

def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask 
        # if borders:
        #     import cv2
        #     contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        #     # Try to smooth contours
        #     contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        #     cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1) 

    ax.imshow(img)

In [4]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

# checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_large.pt"
checkpoint = "/home/killian/sam2/checkpoints/sam2.1_hiera_tiny.pt"
# model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"
# predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))

sam2 = build_sam2(model_cfg, checkpoint, device="cuda", apply_postprocessing=False)

mask_generator = SAM2AutomaticMaskGenerator(sam2)
# with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    # predictor.set_image("/home/killian/sam2")
    # masks, _, _ = predictor.predict("0")
    # 🔹 Chemins vers le modèle et la configuration
#checkpoint = "./checkpoints/sam2_hiera_large.pt"
#model_cfg = "sam2_hiera_l.yaml"


Hyperparamètres à régler

DataLoader

In [5]:
from torch.utils.data import Dataset
import glob
import os
from PIL import Image
import numpy as np
import torch

# On définit le DataLoader
class DataLoaderSegmentation(Dataset):
    def __init__(self, folder_path, transforms=None):
        super(DataLoaderSegmentation, self).__init__()
        self.img_files = glob.glob(os.path.join(folder_path, '*.jpg'))
        # print(self.img_files)
        self.mask_files = []
        for img_path in self.img_files:
            self.mask_files.append(os.path.join(folder_path, f'{img_path[:-4]}_mask.png'))
        
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_files[index]
        mask_path = self.mask_files[index]
        image = Image.open(img_path)
        image = np.array(image.convert("RGB"))
        label = Image.open(mask_path)
        label = np.array(label.convert("RGB"))
        # if self.transforms:
        #     image = self.transforms(torch.from_numpy(image).float())
                 
        return image, torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

permet de georéférencé nos images

In [6]:
import os
import tempfile

import kornia.augmentation as K
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.transforms as T

#from torchgeo.datasets import EuroSAT100
from torchgeo.datasets import RasterDataset, stack_samples
from torchgeo.samplers import GridGeoSampler, RandomGeoSampler
from torchgeo.models import ResNet18_Weights, resnet18
from torchgeo.transforms import AugmentationSequential

import matplotlib.pyplot as plt
torch.manual_seed(0)

ModuleNotFoundError: No module named 'torchgeo'

In [7]:
from torch.utils.data import Dataset
import glob
#on définit le datatloader
class DataLoaderSegmentation(Dataset):
    def __init__(self, folder_path, transforms=None):
        super(DataLoaderSegmentation, self).__init__()
        self.img_files = glob.glob(os.path.join(folder_path,'*.jpg'))
        # print(self.img_files)
        self.mask_files = []
        for img_path in self.img_files:
            self.mask_files.append(os.path.join(folder_path,f'{img_path[:-4]}_mask.png') )
        
        self.transforms = transforms

    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            image = Image.open(img_path)
            image = np.array(image.convert("RGB"))
            label = Image.open(mask_path)
            label = np.array(label.convert("RGB"))
            # if self.transforms:
            #     image = self.transforms(torch.from_numpy(image).float())
                 
            return image, torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)



Dataset

In [8]:
import os
import torchvision.transforms as T

MEANS = [122.39, 118.23, 98.1]
SDS = [39.81, 37.33, 33.04]

test_transform = T.Compose([T.ConvertImageDtype(torch.float32),
                            T.Normalize(MEANS,SDS),])


# dataset = DataLoaderSegmentation('/home/killian/Annotations/Annotations', transforms=test_transform)
dataset = DataLoaderSegmentation('/home/killian/Annotations/Annotations')

img, mask = dataset.__getitem__(0)

In [9]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset=dataset, batch_size=1)

In [10]:
#Hyperparametres
model = SAM2AutomaticMaskGenerator(
    model=sam2,
    points_per_side=20,  # Plus de points pour capturer les details
    points_per_batch=30,  # Augmenter pour calculer le nbre de points pris en meme temps (/!\ GPU)
    pred_iou_thresh=0.7,  # Reduire pour accepter plus de mask
    stability_score_thresh=0.80,  # Rzduire pour ne pas exclure trop de mask
    stability_score_offset=0.8,
    crop_n_layers=2,  # Ammeliore la segmentation des petites structures
    box_nms_thresh=0.80,  # Eviter la suppression excessive de petite structure
    crop_n_points_downscale_factor=2,  # Adapter aux images a haute resolution
    min_mask_region_area=15.0,  # Conserver plus de petits objets
    use_m2m=True,  # Mode avancé 
)

In [11]:
#on definit une fonction pour concatener tout nos mask 
def merge_preds(pred):
    res = torch.stack([torch.Tensor(mask['segmentation']) for mask in pred])
    res.shape
    res = res.any(dim=0)
    return res

Modele run:

In [12]:
import os
import csv
import torch
import torchmetrics
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Définition du device
device = "cuda"

# Définition des métriques
miou = torchmetrics.JaccardIndex(task='binary', num_classes=1).to(device)
precision = torchmetrics.Precision(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
f1_score = torchmetrics.F1Score(task='binary').to(device)

# Dossier de sortie
output_dir = "/home/killian/sam2/predictions"
os.makedirs(output_dir, exist_ok=True)

# Fichier CSV pour les métriques globales
csv_file = os.path.join(output_dir, "sam2_results.csv")
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "IoU", "Precision", "Recall", "F1_Score", "Num_Masks", "Num_Masks_Label"])

# Fichier CSV pour les mesures des masques
csv_masks_file = os.path.join(output_dir, "mask_measurements.csv")
with open(csv_masks_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "Mask_ID", "Centroid_X", "Centroid_Y", "Area", "Equivalent_Diameter"])

size_threshold = 10000  # Seuil pour filtrer les objets trop grands

# Variables pour les moyennes
total_miou = total_precision = total_recall = total_f1 = total_images = 0

for i, (batch, mask) in enumerate(dataloader):
    print(f"Image {i+1}/{len(dataloader)} - Avancement : {i/len(dataloader):.2%}", end='\r')
    batch = batch.to(device)

    for idx, (image, label) in enumerate(zip(batch, mask)):
        image_np = image.cpu().numpy()
        pred = model.generate(image_np)
        
        res_tensor = torch.stack([torch.tensor(m['segmentation'], dtype=torch.bool) for m in pred])
        filtered_tensor = res_tensor[res_tensor.sum(dim=(1, 2)) <= size_threshold]
        res_merge = filtered_tensor.any(dim=0).to(device)
        
        label_np = np.array(label)
        trach = torch.as_tensor(label_np[:, :, 2], dtype=torch.bool, device=device) if label_np.ndim == 3 and label_np.shape[-1] >= 3 else torch.as_tensor(label_np, dtype=torch.bool, device=device)
        
        num_masks_label = (label_np > 0).sum()
        
        # Calcul des métriques
        img_miou = miou(res_merge, trach).item()
        img_precision = precision(res_merge, trach).item()
        img_recall = recall(res_merge, trach).item()
        img_f1 = f1_score(res_merge, trach).item()
        
        total_miou += img_miou
        total_precision += img_precision
        total_recall += img_recall
        total_f1 += img_f1
        total_images += 1
        
        # Sauvegarde des résultats dans le CSV
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([f"Image_{i}_{idx}", img_miou, img_precision, img_recall, img_f1, filtered_tensor.shape[0], num_masks_label])
        
        # Mesure des aires et centroïdes des masques
        for mask_id, mask_pred in enumerate(filtered_tensor):
            mask_np = mask_pred.cpu().numpy().astype(np.uint8)
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_np, connectivity=8)
            
            with open(csv_masks_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                for j in range(1, num_labels):
                    x, y = centroids[j]
                    area = stats[j, cv2.CC_STAT_AREA]
                    equivalent_diameter = np.sqrt(4 * area / np.pi)
                    writer.writerow([f"Image_{i}_{idx}", mask_id, x, y, area, equivalent_diameter])
        
        # Sauvegarde des images
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.title("Prédiction")
        plt.imshow(res_merge.cpu().numpy(), cmap='gray')
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.title("Label")
        plt.imshow(trach.cpu().numpy(), cmap='gray')
        plt.axis('off')
        
        plt.savefig(os.path.join(output_dir, f"Image_{i}_{idx}.png"))
        plt.close()

if total_images > 0:
    mean_miou = total_miou / total_images
    mean_precision = total_precision / total_images
    mean_recall = total_recall / total_images
    mean_f1 = total_f1 / total_images
    
    print(f"mIoU moyen: {mean_miou:.4f}")
    print(f"Précision moyenne: {mean_precision:.4f}")
    print(f"Rappel moyen: {mean_recall:.4f}")
    print(f"F1-Score moyen: {mean_f1:.4f}")


/tmp/ipykernel_2964631/3911576521.py:51: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  label_np = np.array(label)


mIoU moyen: 0.6046ancement : 99.00%
Précision moyenne: 0.7297
Rappel moyen: 0.7681
F1-Score moyen: 0.7306


Modèle de seullage avec Opencv

In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchmetrics
import cv2

# Définition du device
device = "cuda"

# Définition des métriques
miou = torchmetrics.JaccardIndex(task='binary', num_classes=1).to(device)
precision = torchmetrics.Precision(task='binary').to(device)
recall = torchmetrics.Recall(task='binary').to(device)
f1_score = torchmetrics.F1Score(task='binary').to(device)

# Dossier de sortie
output_dir = "/home/killian/Annotations/Seuillage"
os.makedirs(output_dir, exist_ok=True)

# Fichier CSV pour les métriques globales
csv_file = os.path.join(output_dir, "seuillage_results.csv")
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "IoU", "Precision", "Recall", "F1_Score", "Num_Masks", "Num_Masks_Label"])

# Fichier CSV pour les mesures des masques
csv_masks_file = os.path.join(output_dir, "mask_measurements.csv")
with open(csv_masks_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_ID", "Mask_ID", "Centroid_X", "Centroid_Y", "Area", "Equivalent_Diameter"])

# Variables pour les moyennes
total_miou = total_precision = total_recall = total_f1 = total_images = 0

for idx, (image, mask) in enumerate(dataloader):
    print(f"{idx}/{len(dataloader)}", end='\r')
    img_name = dataset.img_files[idx].split("/")[-1]
    
    image = image.squeeze(0).cpu().numpy()
    
    mask = mask.squeeze(0).cpu().numpy()
    mask = mask[:,:,2] # keep only tracheid labels
    
    gray = np.mean(image, axis=2) if image.ndim == 3 else image
    _, binary = cv2.threshold(gray.astype(np.uint8), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    processed = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    processed = cv2.morphologyEx(processed, cv2.MORPH_CLOSE, kernel)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(processed, connectivity=8)
    
    mask_bin = (mask > 0).astype(np.uint8)
    
    processed_bin = (processed > 0).astype(np.uint8)
    
    res_merge = torch.tensor(processed_bin, dtype=torch.bool, device=device)
    
    trach = torch.tensor(mask_bin, dtype=torch.bool, device=device)
    
    img_miou = miou(res_merge, trach).item()
    img_precision = precision(res_merge, trach).item()
    img_recall = recall(res_merge, trach).item()
    img_f1 = f1_score(res_merge, trach).item()
    
    total_miou += img_miou
    total_precision += img_precision
    total_recall += img_recall
    total_f1 += img_f1
    total_images += 1
    
    num_masks_label = (mask_bin > 0).sum()
    
    # Sauvegarde des mesures des 3, 640, 640)masques
    with open(csv_masks_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        for i in range(1, num_labels):
            x, y = centroids[i]
            area = stats[i, cv2.CC_STAT_AREA]
            equivalent_diameter = np.sqrt(4 * area / np.pi)
            writer.writerow([img_name, i, x, y, area, equivalent_diameter])
    
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Image d'origine")
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title("Prédiction")
    pred_image = res_merge.cpu().numpy()
    for centroid in centroids:
        plt.scatter(centroid[0], centroid[1], color='red', s=50, marker='x')
    plt.imshow(pred_image, cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.title("Label")
    plt.imshow(trach.cpu().numpy(), cmap='gray')
    plt.axis('off')
    
    plt.savefig(os.path.join(output_dir, f"Image_{idx}.png"))
    plt.close()
    
if total_images > 0:
    print(f"mIoU moyen: {total_miou / total_images:.4f}")
    print(f"Précision moyenne: {total_precision / total_images:.4f}")
    print(f"Rappel moyen: {total_recall / total_images:.4f}")
    print(f"F1-Score moyen: {total_f1 / total_images:.4f}")


mIoU moyen: 0.5587
Précision moyenne: 0.7562
Rappel moyen: 0.6850
F1-Score moyen: 0.6918


In [32]:
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS>=178956970

extraire_bandes_images('/home/killian/data2025/11478')
extraire_bandes_images('/home/killian/data2025/13823')
extraire_bandes_images('/home/killian/data2025/15485')
extraire_bandes_images('/home/killian/data2025/15492')
extraire_bandes_images('/home/killian/data2025/TGV4')
extraire_bandes_images('/home/killian/data2025/TGV5')

Bande ech1 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB3_ech1.tif
Bande ech2 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB3_ech2.tif
Bande ech3 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB3_ech3.tif
Bande ech1 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB1_ech1.tif
Bande ech2 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB1_ech2.tif
Bande ech3 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB1_ech3.tif
Bande ech1 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB2_ech1.tif
Bande ech2 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB2_ech2.tif
Bande ech3 extraite et enregistrée : /home/killian/data2025/11478/bandes_images/X200_11478_PB2_ech3.tif
Bande ech1 extraite et enregistrée : /home/killian/data2025/1147